In [62]:
"""This module contains functions for extracting statements from text by
feeding the full json schema to ChatGPT."""

'This module contains functions for extracting statements from text by\nfeeding the full json schema to ChatGPT.'

In [63]:
# import libraries
import json
import random
import pandas as pd
from indra.statements.io import stmt_from_json
from tqdm import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm
from indra_gpt.api import run_openai_chat
from indra_gpt.constants import JSON_SCHEMA
import openai.error
from indra.statements import Agent
from indra.ontology.standardize import standardize_agent_name

In [64]:
# import functions
from indra_gpt.run_statement_json_extraction import gpt_stmt_json
from indra_gpt.run_statement_json_extraction import process_indra_object
from indra_gpt.run_statement_json_extraction import gpt_stmt_json

In [65]:
if __name__ == "__main__":
    main("indra_benchmark_corpus_all_correct.json") # run main function here

Extracting:   0%|                                                                                              | 0.00/50.0 [00:00<?, ?statement/s]WARNING: [2023-09-18 16:04:06] indra.statements.statements - Invalid activity type: acetylation
Extracting:   4%|███▍                                                                                   | 2.00/50.0 [00:06<02:46, 3.48s/statement]WARNING: [2023-09-18 16:04:13] indra.statements.statements - Invalid activity type: phosphorylation-dependent interaction
Extracting:  10%|████████▋                                                                              | 5.00/50.0 [00:16<02:27, 3.29s/statement]WARNING: [2023-09-18 16:04:22] indra.statements.statements - Invalid activity type: autoubiquitination
Extracting:  12%|██████████▍                                                                            | 6.00/50.0 [00:20<02:25, 3.32s/statement]WARNING: [2023-09-18 16:04:26] indra.statements.statements - Invalid activity type: None
Extrac

Done.


In [148]:
"""Checking For Accuracy"""

'Checking For Accuracy'

In [69]:
df = pd.read_table('statement_json_extraction_results.tsv') # transform tsv file back to dataframe

In [125]:
for ix, row in df.iterrows():
    original_stmt_json = json.loads(row.input)
    print(original_stmt_json['type'])

Acetylation
Acetylation
Acetylation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation
Activation


In [144]:
"""For Output Generated By ChatGPT"""

'For Output Generated By ChatGPT'

In [126]:
for ix, row in df.iterrows():
    gpt_stmt_json = json.loads(row.generated_json_object)
    print(gpt_stmt_json['type'])

Activation
Activation
Activation
IncreaseAmount
Activation
Activation
Activation
Activation
RegulateActivity
Activation
Activation
Activation
IncreaseAmount
Inhibition
Activation
Activation
Activation
IncreaseAmount
Modification
Activation
Activation
Activation
Inhibition
Activation
Activation
Activation
Activation
Activation
Activation
Modification
Activation
Activation
Activation
Activation
Activation
Inhibition
Activation
Inhibition
Activation
Activation
Inhibition
Activation
Activation
Activation
Activation
Activation
Activation
Inhibition
Activation
Activation


In [145]:
"""For Inputted JSON Object"""

'For Inputted JSON Object'

In [147]:
for ix, row in df.iterrows():
    original_stmt_json = json.loads(row.input)
    if original_stmt_json['type'] == 'Acetylation':
        enz = Agent._from_json(original_stmt_json['enz'])  # load the agent
        print(enz.to_json()) # get db refs
    else:
        subj = Agent._from_json(original_stmt_json['subj'])  # load the agent
        print(subj.to_json()) # get db refs

OrderedDict([('name', 'C5'), ('db_refs', {'TEXT': 'C5a', 'NCIT': 'C20487', 'UP': 'P01031', 'HGNC': '1331'})])
OrderedDict([('name', 'KANSL1'), ('db_refs', {'UP': 'Q7Z3B3', 'HGNC': '24565', 'TEXT': 'MSL1v1'})])
OrderedDict([('name', 'EP300'), ('db_refs', {'UP': 'Q09472', 'HGNC': '3373', 'TEXT': 'p300'})])
OrderedDict([('name', 'BMP'), ('db_refs', {'FPLX': 'BMP', 'TEXT': 'BMP'})])
OrderedDict([('name', 'Caspase'), ('db_refs', {'FPLX': 'Caspase', 'TEXT': 'caspase'})])
OrderedDict([('name', 'E3_Ub_ligase'), ('db_refs', {'FPLX': 'E3_Ub_ligase', 'TEXT': 'E3'})])
OrderedDict([('name', 'ERK'), ('db_refs', {'FPLX': 'ERK', 'TEXT': 'MAPK'})])
OrderedDict([('name', 'IL12'), ('db_refs', {'FPLX': 'IL12', 'TEXT': 'IL-12'})])
OrderedDict([('name', 'PDGF'), ('db_refs', {'FPLX': 'PDGF', 'TEXT': 'PDGF'})])
OrderedDict([('name', 'TCR'), ('db_refs', {'FPLX': 'TCR', 'TEXT': 'TCR'})])
OrderedDict([('name', 'TCR'), ('db_refs', {'FPLX': 'TCR', 'TEXT': 'TCR'})])
OrderedDict([('name', 'TGFB'), ('db_refs', {'FPLX

In [143]:
"""For Output Generated By ChatGPT"""

'For Output Generated By ChatGPT'

In [141]:
for ix, row in df.iterrows():
    gpt_stmt_json = json.loads(row.generated_json_object)
    if gpt_stmt_json['type'] == 'Modification':
        enz = Agent._from_json(gpt_stmt_json['enz'])  # load the agent
        print(enz.to_json()) # get db refs
    else:
        subj = Agent._from_json(gpt_stmt_json['subj'])  # load the agent
        print(subj.to_json()) # get db refs

OrderedDict([('name', 'C5a'), ('db_refs', {'TEXT': 'C5a'})])
OrderedDict([('name', 'MOF and MSL1v1 complex'), ('db_refs', {'TEXT': 'MOF and MSL1v1 complex'})])
OrderedDict([('name', 'c-Abl'), ('db_refs', {'UP': 'P00519', 'HGNC': '76', 'TEXT': 'c-Abl'})])
OrderedDict([('name', 'BMP'), ('db_refs', {'TEXT': 'BMP'})])
OrderedDict([('name', 'Caspase'), ('db_refs', {'TEXT': 'caspase'})])
OrderedDict([('name', 'hMDMX p60'), ('db_refs', {'TEXT': 'hMDMX p60'})])
OrderedDict([('name', 'HGF/c-Met signaling'), ('db_refs', {'TEXT': 'HGF/c-Met signaling'})])
OrderedDict([('name', 'IL-18'), ('db_refs', {'TEXT': 'IL-18'})])
OrderedDict([('name', 'LRP1'), ('db_refs', {'HGNC': '6693', 'TEXT': 'LRP1'})])
OrderedDict([('name', 'TCR'), ('db_refs', {'TEXT': 'TCR'})])
OrderedDict([('name', 'TLR2'), ('db_refs', {'TEXT': 'TLR2', 'HGNC': '11850', 'UP': 'O60603', 'NCIT': 'C18079'})])
OrderedDict([('name', 'FFA'), ('db_refs', {'TEXT': 'FFA'})])
OrderedDict([('name', 'ENTPD1'), ('db_refs', {'TEXT': 'ENTPD1', 'HGNC

In [150]:
"""Check Number of Accurate Statement Types"""

'Check Number of Accurate Statement Types'

In [153]:
counter = 0
for ix, row in df.iterrows():
    if original_stmt_json['type'] == gpt_stmt_json['type']:
        print("original: " + str(original_stmt_json['type']), "generated: " + str(gpt_stmt_json['type']))
        counter+=1
print("Number of Correct Statement Types: " + str(counter))

original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: Activation generated: Activation
original: A